# Human VS AI Tic Tac Toe 🤖.

<br>



<p align="center">
  <img width="700" height="400" src="https://github.com/AxelDucamp/PROJET_Tic_Tac_Toe/blob/main/TicTacToe_img.jpg?raw=true">
</p>


<br>
<br>

# Le projet :

Ce projet a pour objectif la création d'une ia permettant de jouer contre celle-ci au jeu tic tac toe. Le modèle a été entraîné sur Google Colab.
Vous trouverez à la fin de ce notebook une démonstration de moi jouant contre mon ia (spoiler alert : l'ia a été vaincu..🤖🤖).



In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,TimeDistributed,Dropout
import time
import builtins  
from tabulate import tabulate

In [12]:
df = pd.read_csv('/content/Tic tac initial results.csv')
columns = df.columns
Impute = SimpleImputer(strategy='constant',fill_value = 9,missing_values="?")
df = Impute.fit_transform(df)
df = pd.DataFrame(df,columns=columns)
df.head(10)

,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,9,9,9,loss
1,4,7,2,6,9,9,9,win
2,0,8,1,6,5,9,9,draw
3,4,7,2,3,9,9,9,draw
4,0,4,2,1,9,9,9,win
5,8,5,6,9,9,9,9,loss
6,0,2,7,5,6,9,9,draw
7,4,1,3,8,9,9,9,draw
8,2,6,7,9,9,9,9,loss
9,4,0,8,9,9,9,9,win


In [115]:
liste_res = list()
columns = df.columns[:-1]
for i in np.arange(0,len(df),2):
  for z in range(len(columns)):
    if z == 0:
      liste_temp = list()
      liste_temp.append(df[columns[z]].iloc[i])
      liste_temp.append(df[columns[z]].iloc[i+1])
      liste_res.append(liste_temp)
      liste_memoire = liste_temp.copy()
    else:
        liste_memoire.append(df[columns[z]].iloc[i])
        liste_res.append(liste_memoire.copy())
        
        liste_memoire.append(df[columns[z]].iloc[i+1])
        liste_res.append(liste_memoire.copy())

In [118]:
liste_res[:14]

[[0, 4],
 [0, 4, '8'],
 [0, 4, '8', '7'],
 [0, 4, '8', '7', '1'],
 [0, 4, '8', '7', '1', '2'],
 [0, 4, '8', '7', '1', '2', '3'],
 [0, 4, '8', '7', '1', '2', '3', '6'],
 [0, 4, '8', '7', '1', '2', '3', '6', 9],
 [0, 4, '8', '7', '1', '2', '3', '6', 9, 9],
 [0, 4, '8', '7', '1', '2', '3', '6', 9, 9, 9],
 [0, 4, '8', '7', '1', '2', '3', '6', 9, 9, 9, 9],
 [0, 4, '8', '7', '1', '2', '3', '6', 9, 9, 9, 9, 9],
 [0, 4, '8', '7', '1', '2', '3', '6', 9, 9, 9, 9, 9, 9],
 [0, 4]]

In [117]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = 13
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=10)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(liste_res)

In [119]:
X = predictors
y = label

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [120]:
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [153]:
model = Sequential([
        LSTM(10,input_shape=(1,12),return_sequences=True),
        LSTM(20,return_sequences=(False)),
        Dropout(0.15),
        Dense(16,activation='relu'),
        Dense(10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [154]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5000,batch_size=32)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
396/396 [==============================] - 3s 7ms/step - loss: 0.9244 - accuracy: 0.6770 - val_loss: 0.9868 - val_accuracy: 0.6713
Epoch 2502/5000
396/396 [==============================] - 3s 7ms/step - loss: 0.9295 - accuracy: 0.6806 - val_loss: 0.9857 - val_accuracy: 0.6748
Epoch 2503/5000
396/396 [==============================] - 3s 7ms/step - loss: 0.9251 - accuracy: 0.6788 - val_loss: 0.9856 - val_accuracy: 0.6726
Epoch 2504/5000
396/396 [==============================] - 3s 7ms/step - loss: 0.9245 - accuracy: 0.6800 - val_loss: 0.9863 - val_accuracy: 0.6754
Epoch 2505/5000
396/396 [==============================] - 3s 7ms/step - loss: 0.9242 - accuracy: 0.6822 - val_loss: 0.9857 - val_accuracy: 0.6729
Epoch 2506/5000
396/396 [==============================] - 3s 7ms/step - loss: 0.9249 - accuracy: 0.6816 - val_loss: 0.9858 - val_accuracy: 0.6751
Epoch 2507/5000
396/396 [==============================]

In [156]:
def Play_Tic_Tac_Toe(Game_seq):
  Game_seq_list = pad_sequences([Game_seq], maxlen=max_sequence_len-1, padding='pre')
  Game_seq_list = Game_seq_list.reshape(1,1,12)
  predicted = np.argmax(model.predict(Game_seq_list, verbose=0))
  return predicted

In [157]:
def list_tab(value,L1,L2,L3,figure):
    if value > 2 and value < 6:
      if value == 3:
        index = 0
      elif value == 4:
        index = 1
      else:
        index = 2
      L2[index] = figure
    elif value >= 0 and value < 3:
      L1[value] = figure
    elif value >= 6 and value < 9:
      if value == 6:
        index = 0
      elif value == 7:
        index = 1
      else:
        index = 2
      L3[index] = figure
    return L1,L2,L3

In [158]:
def verif(liste_val):
  stop = False
  if len(liste_val) > len(set(liste_val)):
    stop = True
  return stop

In [159]:
def verif_gagnant(L1,L2,L3):
  symbol = ['X','O']
  gagnant = 0
  qui = ""
  for S in symbol:
    if L1[0] == L2[0] == L3[0] == S:
      gagnant = 1
      qui = S
    elif L1[1] == L2[1] == L3[1] == S:
      gagnant = 1
      qui = S
    elif L1[2] == L2[2] == L3[2] == S:
      gagnant = 1
      qui = S
    elif L1[0] == L2[1] == L3[2] == S:
      gagnant = 1
      qui = S
    elif L1[2] == L2[1] == L3[0] == S:
      gagnant = 1
      qui = S
    elif L1[0] == L1[1] == L1[2] == S:
      gagnant = 1
      qui = S
    elif L2[0] == L2[1] == L2[2] == S:
      gagnant = 1
      qui = S
    elif L3[0] == L3[1] == L3[2] == S:
      gagnant = 1
      qui = S
  compteur = 0
  if gagnant != 1:
    for l in [L1,L2,L3]:
      for elem in l:
        if elem != ' ':
          compteur += 1
    if compteur == 9:
      qui = "nobody"
  return gagnant , qui

In [177]:
def START_TIC_TAC_TOE():
  end_game = 0
  L1 = [" ", " ", " "]
  L2 = [" ", " ", " "]
  L3 = [" ", " ", " "]
  Game_seq = list()
  another = 0
  print('INFO : ')
  print('-'*30)
  print(tabulate([["0", "1", "2"], ["3", "4", "5"], ["6", "7", "8"]], tablefmt="grid"))
  print('-'*30)
  while end_game != 1:
    input_player = int(builtins.input("It is your turn ! :D   :"))
    Game_seq.append(input_player)
    figure = "X"
    L1,L2,L3 = list_tab(input_player,L1,L2,L3,figure)
    print(tabulate([L1, L2, L3], tablefmt="grid"))
    time.sleep(2)
    computer_player = Play_Tic_Tac_Toe(Game_seq)
    Game_seq.append(computer_player)
    figure = "O"
    L1,L2,L3 = list_tab(computer_player,L1,L2,L3,figure)
    stop = verif(Game_seq)
    gagnant = verif_gagnant(L1,L2,L3)
    if stop:
      print('Oh nooo there was an error')
      print('..:(')
      val_try_again = builtins.input('Do you want to try again? {y/n} :')
      if val_try_again == "y":
        START_TIC_TAC_TOE()
        another = 1
      end_game = 1
    if not stop or gagnant[0] != 1:
      if gagnant[1] == "nobody":
        end_game = 1
        val_try_again = builtins.input('Oh no! nobody won :( Do you want to try again? {y/n} :')
        if val_try_again == 'y':
          START_TIC_TAC_TOE()
          another = 1
          end_game = 1
      else:
        if another == 0:
          print(tabulate([L1, L2, L3], tablefmt="grid"))
    if gagnant[0] == 1:
      if gagnant[1] == "X":
        print('You won !')
      else:
        print('Try Again.. ;)')
      end_game = 1

In [178]:
START_TIC_TAC_TOE()

INFO : 
------------------------------
+---+---+---+
| 0 | 1 | 2 |
+---+---+---+
| 3 | 4 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
------------------------------
It is your turn ! :D   :2
+--+--+---+
|  |  | X |
+--+--+---+
|  |  |   |
+--+--+---+
|  |  |   |
+--+--+---+
+--+---+---+
|  |   | X |
+--+---+---+
|  | O |   |
+--+---+---+
|  |   |   |
+--+---+---+
It is your turn ! :D   :8
+--+---+---+
|  |   | X |
+--+---+---+
|  | O |   |
+--+---+---+
|  |   | X |
+--+---+---+
+--+---+---+
|  |   | X |
+--+---+---+
|  | O | O |
+--+---+---+
|  |   | X |
+--+---+---+
It is your turn ! :D   :3
+---+---+---+
|   |   | X |
+---+---+---+
| X | O | O |
+---+---+---+
|   |   | X |
+---+---+---+
+---+---+---+
|   | O | X |
+---+---+---+
| X | O | O |
+---+---+---+
|   |   | X |
+---+---+---+
It is your turn ! :D   :7
+---+---+---+
|   | O | X |
+---+---+---+
| X | O | O |
+---+---+---+
|   | X | X |
+---+---+---+
+---+---+---+
|   | O | X |
+---+---+---+
| X | O | O |
+---+---+---+
| O | X